In [1]:
import matplotlib.pyplot as plt
import gzip, os, sys
import numpy as np
from scipy.stats import multivariate_normal

if sys.version_info[0] == 2:
    from urllib import urlretrieve
else:
    from urllib.request import urlretrieve

In [2]:
def load_mnist_images(filename):
    # if not os.path.exists(filename):
    #     download(filename)
    with gzip.open(filename, 'rb') as f:
        data = np.frombuffer(f.read(), np.uint8, offset=16)
    data = data.reshape(-1,784)
    return data
# 加载标签
def load_mnist_labels(filename): 
    # if not os.path.exists(filename):
    #     download(filename)
    with gzip.open(filename, 'rb') as f:
        data = np.frombuffer(f.read(), np.uint8, offset=8)
    return data

In [3]:
train_data = load_mnist_images('train-images-idx3-ubyte.gz')
train_labels = load_mnist_labels('train-labels-idx1-ubyte.gz')
test_data = load_mnist_images('t10k-images-idx3-ubyte.gz')
test_labels = load_mnist_labels('t10k-labels-idx1-ubyte.gz')

In [6]:
print(train_data.shape)

(60000, 784)


In [7]:
def show_digit(x, label):
     plt.axis('off')
     plt.imshow(x.reshape((28,28)), cmap=plt.cm.gray)
     plt.title('Label ' + str(label))
     plt.show()
     return

In [8]:
import time
from sklearn.neighbors import BallTree
## Build nearest neighbor structure on training data
t_before = time.time()
ball_tree = BallTree(train_data[3000:4000,:])
t_after = time.time()
# Compute training time
t_training = t_after - t_before
print("Time to build data structure (seconds): ", t_training)
## Get nearest neighbor predictions on testing data
test_data1=test_data[500,:]
test_labels1=test_labels[500]

t_before = time.time()
test_data1 = np.expand_dims(test_data1,0)
test_neighbors = ball_tree.query(test_data1, k=100, return_distance=False)
test_neighbors = np.squeeze(test_neighbors)
# test_neighbors = np.squeeze(ball_tree.query(test_data1, k=1, return_distance=False))
test_predictions = train_labels[test_neighbors]
t_after = time.time()
## Compute testing time
t_testing = t_after - t_before
print("Time to classify test set (seconds): ", t_testing)
# evaluate the classifier
t_accuracy = sum(test_predictions == test_labels1) / 100 #float(len(test_labels1))
# t_accuracy
print("accuracy: ", t_accuracy)
0.96909999999999996

import pandas as pd
import seaborn as sn
from sklearn import metrics
cm = metrics.confusion_matrix(test_labels,test_predictions)
df_cm = pd.DataFrame(cm, range(10), range(10))
sn.set(font_scale=1.2)# for label size
sn.heatmap(df_cm, annot=True,annot_kws={"size": 16}, fmt="g")

Time to build data structure (seconds):  0.025021076202392578
Time to classify test set (seconds):  0.0020036697387695312
accuracy:  0.07


ValueError: Found input variables with inconsistent numbers of samples: [10000, 100]